In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import os
import re
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision


In [37]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [38]:
# Create training dataset
# Set up the data directories
data_dir = 'Test_Images'
train_dir = os.path.join(data_dir, 'train')

# Define a function to get the labels from the image filenames
def get_label(filename):
    match = re.search(r'\d+\.?\d*', filename)
    if match:
        return float(match.group())
    else:
        return None

# Define a list to store the image filenames and labels
train_data = []

# Iterate over the training images and add them to the list
for filename in os.listdir(train_dir):
    label = get_label(filename)
    if label is not None:
        train_data.append([os.path.join(train_dir, filename), label])

# Convert the list to a dataframe
train_df = pd.DataFrame(train_data, columns=['filename', 'label'])

# Save the dataframe to a CSV file
train_df.to_csv(os.path.join(data_dir, 'train.csv'), index=False)

In [35]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.data = self._load_data(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_path, label = self.data[index]
        image = Image.open(image_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def _load_data(self, csv_file):
    # Load and preprocess data from the CSV file
    # Example: Assuming the CSV file has two columns representing image paths and labels
        data = []
        with open(csv_file, 'r') as file:
            lines = file.readlines()
            for line in lines[1:]:  # Skip the header row
                # Split the line by comma or any other appropriate delimiter
                image_path, label = line.strip().split(',')
                data.append((image_path, float(label)))  # Parse the label as float
        return data


# Example usage:
csv_file = 'Test_Images/train.csv'  # Replace with the path to your CSV file
image_folder = 'Test_Images/train'  # Replace with the path to your image folder
dataset = CustomImageDataset(csv_file, image_folder, transform=torchvision.transforms.ToTensor())

# Access individual samples
image, label = dataset[0]
print(image.shape, label)

torch.Size([3, 1080, 1920]) 100.0


In [85]:
# Set up the data
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
csv_file = 'Test_Images/train.csv'  # Replace with the path to your CSV file
image_folder = 'Test_Images/train'  # Replace with the path to your image folder

dataset = CustomImageDataset(csv_file, image_folder, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [79]:
# Define the loss function and optimizer
model = CNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [92]:
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    
    # Training phase
    model.train()  # Set the model to training mode
    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        labels = labels.unsqueeze(1)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.6f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        val_loss = 0.0
        val_samples = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            labels = labels.unsqueeze(1)
            loss = criterion(outputs, labels.float())
            val_loss += loss.item() * inputs.size(0)
            val_samples += inputs.size(0)
        
        average_val_loss = val_loss / val_samples
        print('Epoch %d - Validation Loss: %.6f' % (epoch + 1, average_val_loss))
        
    print('Epoch %d completed.' % (epoch + 1))

torch.save(model, 'trained_model')

Epoch 1 - Validation Loss: 156.253153
Epoch 1 completed.
Epoch 2 - Validation Loss: 156.253153
Epoch 2 completed.
Epoch 3 - Validation Loss: 156.253153
Epoch 3 completed.
Epoch 4 - Validation Loss: 156.253153
Epoch 4 completed.
Epoch 5 - Validation Loss: 156.253153
Epoch 5 completed.


In [90]:
model = torch.load('trained_model')  # Replace 'trained_model.pth' with the path to your trained model file

# Load and preprocess the input image
input_image = Image.open('Test_Images\drop_s100_v15_r0.5_str6_pos1.png_edged.png')  # Replace 'input_image.jpg' with the path to your input image file
input_image = input_image.convert('RGB')
input_tensor = transform(input_image)

# input_tensor = torchvision.transforms.ToTensor()(input_image).unsqueeze(0)  # Preprocess the image and add a batch dimension
input_tensor.shape



torch.Size([3, 64, 64])

In [91]:
# Set the model in evaluation mode
model.eval()

# Make a prediction
with torch.no_grad():
    output = model(input_tensor)

# Convert the output to a readable format
predicted_value = output.item()

# Print the predicted value
print('Predicted value:', predicted_value)

Predicted value: 87.73915100097656
